In [ ]:
!pip install openai "numpy<2.0" faiss-cpu scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 20.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you 

In [ ]:
!pip install sentence-transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import numpy as np
import faiss
from openai import OpenAI
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import time, re, json

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

In [ ]:
import configparser
import os
from openai import OpenAI

# --- 설정 파일에서 API 키 불러오기 ---

# ConfigParser 객체 생성
config = configparser.ConfigParser(interpolation=None)

# properties 파일 경로 설정
properties_file_path = 'app.properties'

# 파일이 존재하는지 확인 후 읽기
if not os.path.exists(properties_file_path):
    exit(f"오류: 설정 파일 '{properties_file_path}'을(를) 찾을 수 없습니다.")

try:
    config.read(properties_file_path)
    # 'API' 섹션에서 각 키 값 읽어오기
    my_api_key = config.get('API', 'openai.api.key')
    gov_api_key = config.get('API', 'govdata.api.key')
except (configparser.NoSectionError, configparser.NoOptionError) as e:
    exit(f"설정 파일 읽기 오류: {e}")

# OpenAI 클라이언트 초기화
try:
    # 키 값이 비어있는지 확인
    if not my_api_key or my_api_key == 'YOUR_OPENAI_API_KEY_HERE':
        raise ValueError("app.properties 파일에 OpenAI API 키가 설정되지 않았습니다.")

    client = OpenAI(api_key=my_api_key)
except (TypeError, ValueError) as e:
    exit(f"OpenAI 클라이언트 초기화 실패: {e}")


# 정부 데이터 API 키 사용 (필요에 따라 활용)
if not gov_api_key or gov_api_key == 'YOUR_GOV_API_KEY_HERE':
    print("경고: app.properties 파일에 정부 데이터 API 키가 설정되지 않았습니다.")


# (아래부터) 리팩토링 시작하기
**'플러그인 아키텍처(Plugin Architecture)'**를 도입하여 AI 에이전트의 핵심 코드는 그대로 둔 채, 새로운 '도구(Tool)'를 마치 앱처럼 간단하게 추가하고 제거할 수 있는 유연한 구조 설계

In [ ]:
# 파일명: tools/base.py
from abc import ABC, abstractmethod

class ToolBase(ABC):
    @property
    @abstractmethod
    def name(self) -> str:
        """도구의 이름 (AI가 호출할 이름)"""
        pass

    @property
    @abstractmethod
    def description(self) -> str:
        """도구의 기능에 대한 설명"""
        pass

    @property
    @abstractmethod
    def parameters(self) -> dict:
        """도구가 받는 파라미터 명세 (JSON Schema)"""
        pass

    @abstractmethod
    def execute(self, **kwargs) -> str:
        """도구의 실제 로직을 수행하는 메소드"""
        pass

In [ ]:
# 파일명: tool_loader.py
import os
import importlib
import inspect
from tools.base import ToolBase

class ToolLoader:
    def __init__(self, rag_system, user_database, tool_directory: str = "tools"):
        # tool 에 넘길 변수를 설정하는 경우 아래에다 설정
        self.services = {
            "rag_system": rag_system,
            "user_database": user_database,
            "_client": client,
            "_gov_api_key": gov_api_key
        }
        self.tools = self._load_tools(tool_directory)

    def _load_tools(self, tool_directory):
        loaded_tools = []

        if not os.path.isdir(tool_directory):
            print(f"[ToolLoader Error] '{tool_directory}' 디렉토리를 찾을 수 없습니다. 현재 작업 경로는 '{os.getcwd()}' 입니다.")
            return loaded_tools

        print(f"[ToolLoader] '{tool_directory}' 디렉토리에서 도구를 검색합니다...")

        for filename in os.listdir(tool_directory):
            if filename.endswith("_tool.py"):
                module_name = f"{tool_directory}.{filename[:-3]}"

                try:
                    module = importlib.import_module(module_name)
                    print(f"  - 모듈 로드 성공: {module_name}")

                    for attribute_name in dir(module):
                        attribute = getattr(module, attribute_name)
                        if (isinstance(attribute, type) and
                            issubclass(attribute, ToolBase) and
                            attribute is not ToolBase): # ToolBase 자체는 제외

                            # Tool Generator 가 멍청해서 비정상 class를 만드는 경우가 있다
                            # 이 경우 정상적인 parameter 만 가진 도구만 이용하도록 한다
                            if inspect.isabstract(attribute):
                                print(f"  [ToolLoader Warning] '{attribute.__name__}' 클래스는 추상 속성을 모두 구현하지 않아 로드하지 않습니다.")
                                continue # 불완전한 도구는 건너뛰기

                            # 검증된 도구만 이용
                            sig = inspect.signature(attribute.__init__)
                            params = sig.parameters

                            dependencies = {}
                            for param_name in params:
                                if param_name in self.services:
                                    dependencies[param_name] = self.services[param_name]

                            tool_instance = attribute(**dependencies)
                            loaded_tools.append(tool_instance)
                            print(f"    - 도구 등록 완료: {tool_instance.name}")

                except ImportError as e:
                    print(f"[ToolLoader Error] 모듈 '{module_name}'을 import하는 중 오류 발생: {e}")

        if not loaded_tools:
            print("[ToolLoader Warning] 로드된 도구가 없습니다. 'tools' 디렉토리 구조와 파일명(_tool.py)을 확인하세요.")

        return loaded_tools

AIToolbox 내부 Tool들을 개별 python 파일로 리팩토링한다

In [ ]:
# 개별 python 파일 tool 을 기반으로 작동하는 에이전트 코드
from tools.utils.SystemUtils import PrivacyUtils
from tools.tool_generator import ToolGenerationPipeline

class AIAgent:
    def __init__(self, user_id: str, rag_system, user_database):
        self.rag_system = rag_system
        self.user_id = user_id
        self.USER_DB = user_database
        self.user = self.USER_DB.get(user_id, {"user_id": user_id})
        # self.tool_loader = ToolLoader(
        #     rag_system=rag_system,
        #     user_database=user_database,
        #     tool_directory="tools"
        # )
        # self.tools = self.tool_loader.tools

        # # 로드된 도구 목록을 기반으로 available_tools와 api_tools를 자동 생성
        # self.available_tools = {tool.name: tool.execute for tool in self.tools}
        # self.api_tools = [
        #     {"type": "function", "function": {
        #         "name": tool.name,
        #         "description": tool.description,
        #         "parameters": tool.parameters
        #     }} for tool in self.tools
        # ]

        # ToolLoader를 통해 모든 플러그인 도구를 동적으로 로드
        # _reload_tools 로 대체
        self._reload_tools()

        # self.toolbox = AgentToolbox(rag_system)
        # self.user = self.USER_DB.get(user_id, {"user_id": user_id})

        # self.available_tools = { # 실행할 함수 매핑
        #     "search_knowledge_base": self.toolbox.search_knowledge_base,
        #     "fetch_document_from_mcp": self.toolbox.fetch_document_from_mcp,
        #     "validate_document": self.toolbox.validate_document,
        #     "submit_application": self.toolbox.submit_application,
        #     "verify_business_registration": self.toolbox.verify_business_registration,
        #     "synchronize_knowledge_base": self.toolbox.synchronize_knowledge_base,
        # }
        # # OpenAI API에 전달할 도구 명세 정의
        # self.api_tools = [
        #     {"type": "function", "function": {"name": "search_knowledge_base", "description": "사용자 질문과 가장 관련된 정책 정보를 지식 베이스에서 검색합니다.", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "사용자의 원본 질문"}}, "required": ["query"]}}},
        #     {"type": "function", "function": {"name": "fetch_document_from_mcp", "description": "필요한 서류를 MCP를 통해 기관에서 가져옵니다.", "parameters": {"type": "object", "properties": {"document_name": {"type": "string", "description": "가져올 서류의 정확한 이름"}, "user_id": {"type": "string", "description": "요청하는 사용자의 ID"}}, "required": ["document_name", "user_id"]}}},
        #     {"type": "function", "function": {"name": "validate_document", "description": "가져온 서류가 유효한지(예: 유효기간) 검증합니다.", "parameters": {"type": "object", "properties": {"doc_token": {"type": "string", "description": "검증할 서류의 확인 토큰"}, "issue_date_str": {"type": "string", "description": "서류의 발급일자(YYYY-MM-DD 형식)"}}, "required": ["doc_token", "issue_date_str"]}}},
        #     {"type": "function", "function": {"name": "submit_application", "description": "모든 검증된 서류를 모아 최종 목적지에 제출합니다.", "parameters": {"type": "object", "properties": {"doc_tokens": {"type": "array", "items": {"type": "string"}}, "destination": {"type": "string", "description": "제출할 기관 이름"}}, "required": ["doc_tokens", "destination"]}}},
        #     # {"type": "function", "function": {"name": "verify_business_registration", "description": "사용자의 사업자등록 상태가 유효한지를 확인합니다.", "parameters": {"type": "object", "properties": {"user_id": {"type": "string", "description": "상태를 조회할 사용자의 고유 ID(예 : user_id)"}}, "required": ["user_id"]}}},
        #     {"type": "function", "function": {
        #         "name": "verify_business_registration",
        #         "description": "사용자의 사업자등록 상태가 유효한지 확인합니다. 이 도구는 별도의 파라미터 없이 호출하면, 현재 사용자의 정보로 자동 조회됩니다.",
        #         "parameters": {"type": "object", "properties": {}} # 파라미터를 비워서 규약 2를 적용
        #     }},
        #     {"type": "function", "function": {"name": "synchronize_knowledge_base", "description": "외부 소스로부터 RAG 지식 베이스를 최신 상태로 동기화합니다.", "parameters": {"type": "object", "properties": {"filepath": {"type": "string"}}, "required": ["filepath"]}}},
        # ]


    # Tool 을 다시 불러오는 함수
    def _reload_tools(self):
        print("\n[AI Agent] 도구 목록을 새로고침합니다...")

        self.tool_loader = ToolLoader(
            rag_system=self.rag_system,
            user_database=self.USER_DB,
            tool_directory="tools"
        )

        self.tools = self.tool_loader.tools
        self.available_tools = {tool.name: tool.execute for tool in self.tools}
        self.api_tools = [
            {"type": "function", "function": {
                "name": tool.name, "description": tool.description, "parameters": tool.parameters
            }} for tool in self.tools
        ]

        print(f"[AI Agent] 현재 사용 가능한 도구: {[tool.name for tool in self.tools]}")


    # GateKeeper Filter 함수
    # LLM이 사람을 돕도록 System prompt 가 있어 서비스 외 질문에도 답변을 해버린다
    # 이러한 현상을 해결하기 위해 맨 앞에서 서비스 의도 질문인지를 분류해버림
    def _is_query_in_scope(self, query: str) -> bool:
        print("  [Gatekeeper] 사용자 질문의 의도를 분류합니다...")

        # 의도 분류만을 위한 매우 구체적이고 단순한 프롬프트
        system_prompt = """
            당신은 사용자 질문의 핵심 의도가 '대한민국의 행정 또는 금융 신청 업무'와 관련 있는지 판단하는 분류 전문가입니다.
            사용자의 궁극적인 목표가 대출, 지원금, 계좌 개설, 서류 발급 등과 관련 있다면 'YES'입니다.

            **판단 예시:**
            - 질문: "IT 스타트업을 차릴 건데, 사업자금 대출 알려줘." -> YES
            - 질문: "가게 운영자금이 부족해요." -> YES
            - 질문: "청년도약계좌 만들고 싶어요." -> YES
            - 질문: "오늘 날씨 어때?" -> NO
            - 질문: "낚시하는 법 알려줘" -> NO

            다른 어떤 설명도 하지 말고, 오직 'YES' 또는 'NO'로만 대답하세요.
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o", # 또는 더 저렴한 gpt-3.5-turbo 사용 가능
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": query}
                ],
                max_tokens=2, # 답변을 'YES' 또는 'NO'로 제한
                temperature=0.0
            )
            decision = response.choices[0].message.content.strip().upper()
            print(f"  [Gatekeeper] 판단 결과: {decision}")
            return decision == "YES"
        except Exception as e:
            print(f"  [Gatekeeper] 의도 분류 중 오류 발생: {e}")
            return False # 오류 발생 시 보수적으로 접근하여 거절


    # 실제 에이전트 실행
    def run(self, initial_query: str):
        print(f"tool_loader : {self.tool_loader}")
        print(f"tools : {self.tools}")
        print(f"정의된 Tool들 : {self.api_tools}")
        print(f"사용가능 Tool들 : {self.available_tools}")
        print(f"\n{'#'*10} AI Agent Process Start (Query: '{initial_query}') {'#'*10}")

        # LLM 에 민감정보를 던지지 않기 위해 임의의 id를 내부 Database 에서 조회한다
        contextual_query = f"""
            [사용자 ID]
            {self.user.get('user_id', 'N/A')}

            [사용자 요청 사항]
            {initial_query}
        """

        # GateKeeper 에 의해 질문의 정상 여부(서비스 목적에 맞는지) 확인
        if not self._is_query_in_scope(initial_query):
            refusal_message = "죄송합니다. 저는 대한민국의 행정 및 금융 신청을 돕기 위해 설계된 전문 AI 에이전트입니다. 문의하신 내용에 대해서는 답변을 드리기 어렵습니다. '소상공인 대출'이나 '청년도약계좌' 등 도움이 필요한 신청 업무가 있으시다면 말씀해주세요."
            print("\n[AI-Linker 최종 답변]")
            print(refusal_message)
            print(f"\n{'#'*10} AI Agent Process Finished (Out of Scope) {'#'*10}")
            return # 프로세스 즉시 종료

        # --- 이 아래는 '문지기'를 통과한 경우에만 실행됩니다 ---
        messages = [
            {"role": "system",
             "content": """
                ### 역할 정의 ###
                당신은 'AI-Linker'입니다. 당신의 유일한 임무는 '대한민국의 행정 및 금융 신청 업무'를 자동화하여 사용자를 돕는 것입니다.
                당신에게 전달된 모든 사용자 요청은 이미 관련성 검사를 통과했습니다. 당신은 질문의 의도를 의심할 필요 없이, 오직 아래의 업무 수행 계획에 따라 목표를 완수하는 데만 집중하세요.
                당신은 오직 '사용자 ID'를 통해서만 사용자를 식별하며, 절대 실제 개인정보를 묻거나 다루지 않습니다.
                도구를 사용할 때는, 반드시 [사용자 ID] 컨텍스트로 제공된 값을 그대로 사용해야 합니다.
                절대로 임의의 ID나 예시 값을 만들어서 사용하면 안 됩니다.


                ### **[매우 중요한 업무 수행 계획 (SOP)]** ###
                당신은 반드시 다음의 논리적 순서에 따라 단계별로 계획을 세우고 도구를 사용해야 합니다.

                **0. 지식 동기화:**
               - 가장 먼저, `synchronize_knowledge_base` 도구를 사용해 `latest_policies.json` 파일과 당신의 지식을 동기화하여 최신 상태를 유지합니다.

                **1. 정보 검색 단계:**
                - 가장 먼저, 사용자의 질문 의도를 파악하여 `search_knowledge_base` 도구를 사용해 관련 정책 정보를 검색합니다.
                - 만약 검색 결과가 "관련 정보를 찾지 못했습니다" 라면, 더 이상 다른 도구를 사용하지 말고 사용자에게 이 사실을 알리고 프로세스를 종료합니다.

                **2. 사업자 상태 확인:**
                - 먼저, `사용자 ID`를 `verify_business_registration` 도구에 전달하여 국세청 상태를 확인합니다.
                - 상태가 정상이 아니면 프로세스를 중단합니다.

                **3. 서류 수집 및 검증 단계:**
                - 정보 검색에 성공했다면, 결과에 포함된 **`metadata`의 `required_docs` 리스트**를 확인합니다.
                - 리스트에 있는 **모든 서류에 대해**, `fetch_document_from_mcp` 도구를 **하나씩 순서대로 호출**하여 서류를 가져옵니다.
                - 각 서류를 가져온 직후, 즉시 `validate_document` 도구를 사용하여 해당 서류가 유효한지 검증합니다.

                **4. 최종 제출 단계:**
                - 모든 서류의 수집 및 검증이 성공적으로 완료되었다면, 확보한 모든 `doc_token`들을 모아 `submit_application` 도구를 호출하여 최종 제출을 완료합니다.

                **5. [매우 중요] 작업 완료:**
                - 사용자의 모든 요청이 완전히 해결되었다고 판단되면, 반드시 `finish_task` 도구를 호출하여 최종 요약 메시지와 함께 작업을 종료해야 합니다.

                ### **[매우 중요한 출력 형식 규칙]** ###
                - 당신이 도구를 사용해야 한다고 판단했을 때, 다른 자연어 설명은 일절 포함하지 마십시오.
                - 반드시 `tool_calls` JSON 객체 형식으로만 응답해야 합니다.
                """
            },
            # {"role": "user", "content": initial_query}
            {"role": "user", "content": contextual_query} # user_id 로 되어있는 부분을 이용하도록 유도
        ]

        for i in range(7): # 최대 7단계 실행
            print(f"\n--- Agent Step {i+1} ---")

            response = client.chat.completions.create(
                model="gpt-4o", messages=messages, tools=self.api_tools, tool_choice="auto"
            )
            response_message = response.choices[0].message
            messages.append(response_message)

            print(f"responseMsg : {response_message}")

            if not response_message.tool_calls:
                # [개선] AI가 작업을 완료하지 못했다고 판단되면, 자기 개선 로직 실행
                print("  [Thought] 현재 도구로는 이 요청을 완료할 수 없습니다. 새로운 도구가 필요한지 확인합니다.")
                prompt_for_reflection = f"사용자 요청 '{initial_query}'을 해결하기 위해 당신의 현재 도구 목록 {[t.name for t in self.tools]}에 없는 새로운 도구가 필요합니까? 필요하다면 그 도구의 'name', 'description', 'parameters'를 JSON으로, 필요없다면 null을 반환하세요."
                reflection_response = client.chat.completions.create(model="gpt-4o", messages=[{"role": "user", "content": prompt_for_reflection}], response_format={"type": "json_object"})
                new_tool_spec = json.loads(reflection_response.choices[0].message.content)

                if new_tool_spec:
                    print("\n[AI Agent] 필요한 새 도구를 발견했습니다. 도구 생성 파이프라인을 가동합니다...")
                    pipeline = ToolGenerationPipeline(client)
                    success = pipeline.create_and_register_tool(new_tool_spec)

                    if success:
                        self._reload_tools()
                        print("[AI Agent] 새 도구를 장착했습니다. 처음부터 작업을 다시 시도합니다.")
                        continue
                    else:
                        print("[AI Agent] 새 도구 제작에 실패하여, 작업을 중단합니다.")
                else:
                    print("[AI Agent] 추가 도구가 필요 없다고 판단. 최종 답변을 출력합니다.")
                    print(f"  [Final Answer] {response_message.content}")
                return

            # 기존 Tool Calling 로직
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                function_to_call = self.available_tools[function_name]

                # AI가 '작업 완료' 신호를 보낸 경우 -> 완전 종료
                if function_name == "finish_task":
                    summary_args = json.loads(tool_call.function.arguments)
                    print(f"  [Thought] 모든 작업이 완료되었다고 판단했습니다.")
                    print(f"  [Final Answer] {summary_args.get('summary')}")
                    return # 여기서 run 메소드를 완전히 종료

                # parameter 를 넘기지 않는 경우 해결
                # 하드 코딩하는 것이 최선의 방안인가?
                if not function_args :
                    tool_output = function_to_call(user=self.user)
                else :
                    tool_output = function_to_call(**function_args)

                messages.append({"tool_call_id": tool_call.id, "role": "tool", "name": function_name, "content": tool_output})


        print(f"\n{'#'*10} AI Agent Process Finished {'#'*10}")


# **To-do List**
 - For 문 대신 여러개의 LLM 을 도입하여 '역할 분리' (예 :  사용자의 의도를 파악하고 채팅을 수행하는 LLM, 도구를 정의/제작하는 LLM, AiAgent 의 도구 채택+수행을 담당하는 LLM 등)

 - GateKeeper 대신 temperature + P, K 값을 조절해서 할 수 없는지?
 - 정책 심사가 서로 다른 프로세스를 가지고 있는 경우 AI오류를 해결할 수 있는 방법(정밀도 높이는 법, Fine-tuning)
 - 정책 크롤러의 MCP화 (현재는 Mock-up만 존재)
 - 사용자 서류를 자동으로 가져와서 자동으로 토큰화(json화) 하는 서비스 개발(사용자 데이터 더미 -> 분류 + AI 기반 -> USER_DATABASE)
 - tool 제작 AI service 고도화(품질 개선)
 - 쓸모없는 tool의 정리(통합/삭제 등)
 - Token 줄일 수 있는 법(=비용 낮추는 법)
 - 규제 문제
 - API 자동 오케스트로 확장(대기관용)

In [ ]:
from tools.utils.hybriddb import VectorDB_hybrid
from tools.utils.ragsystem import RAG_System
# RAG system 정의
my_rag_system = RAG_System()

# 개선된 시멘틱DB 이용
my_rag_system.set_database = VectorDB_hybrid()

# RAG에 정책 데이터 추가하기 (소상공인 직접대출)
my_rag_system.add_document(
    doc_id="SBC_LOAN",
    content="소상공인 정책자금 직접대출은 소상공인의 자금 조달을 돕습니다. 필수 서류는 사업자등록증명원, 국세납세증명서입니다.",
    metadata={
        "source": "소상공인시장진흥공단", "destination": "소상공인시장진흥공단", "policy_code": "SBC-DIRECT-2025-Q3",
        "required_docs": ["사업자등록증명원", "국세납세증명서"]
    }
)

# RAG에 정책 데이터 추가하기 (청년도약계좌)
my_rag_system.add_document(
    doc_id="YOUTH_ACCOUNT",
    content="""
        청년도약계좌는 청년의 자산 형성을 지원하는 정책형 금융상품입니다.
        만 19세에서 34세 이하 청년이 가입 대상이며, 신청 후 계좌를 개설하고 이용할 수 있습니다.
        청년도약계좌 가입 및 개설을 위해 연령 및 개인소득 요건 확인이 필요하며, 소득확인증명서가 요구됩니다.
        """,
    metadata={
        "source": "서민금융진흥원", "destination": "서민금융진흥원", "policy_code": "YOUTH-LEAP-2025",
        "required_docs": ["소득확인증명서", "연령 확인용 신분증"]
    }
)

  [VectorDB] 시맨틱 검색 모델 'jhgan/ko-sroberta-multitask' 로드 중...
  [Knowledge Base] ADD: 'SBC_LOAN' 문서 추가
  [Knowledge Base] ADD: 'YOUTH_ACCOUNT' 문서 추가


In [ ]:
# RAG에 저장된 정책 정보 보기
my_rag_system.print_documents()


==================== RAG Knowledge Base Full Dump ====================

--- Document ID: SBC_LOAN ---
  [Content] - 소상공인 정책자금 직접대출은 소상공인의 자금 조달을 돕습니다. 필수 서류는 사업자등록증명원, 국세납세증명서입니다.
  [Metadata] - {"source": "소상공인시장진흥공단", "destination": "소상공인시장진흥공단", "policy_code": "SBC-DIRECT-2025-Q3", "required_docs": ["사업자등록증명원", "국세납세증명서"]}

--- Document ID: YOUTH_ACCOUNT ---
  [Content] - 
        청년도약계좌는 청년의 자산 형성을 지원하는 정책형 금융상품입니다.
        만 19세에서 34세 이하 청년이 가입 대상이며, 신청 후 계좌를 개설하고 이용할 수 있습니다.
        청년도약계좌 가입 및 개설을 위해 연령 및 개인소득 요건 확인이 필요하며, 소득확인증명서가 요구됩니다.
        
  [Metadata] - {"source": "서민금융진흥원", "destination": "서민금융진흥원", "policy_code": "YOUTH-LEAP-2025", "required_docs": ["소득확인증명서", "연령 확인용 신분증"]}



In [ ]:
USER_DATABASE = {
        "user_kim": {
            "user_id" : "user_kim",
            "name": "김대표", "business_id": "174-82-00063",
            "resident_id": "950101-1234567"
        }
    }


In [ ]:
agent = AIAgent(
    user_id="user_kim",
    rag_system=my_rag_system,
    user_database=USER_DATABASE
)


[AI Agent] 도구 목록을 새로고침합니다...
[ToolLoader] 'tools' 디렉토리에서 도구를 검색합니다...
  - 모듈 로드 성공: tools.verify_business_registration_tool
    - 도구 등록 완료: verify_business_registration
  - 모듈 로드 성공: tools.submit_application_tool
    - 도구 등록 완료: submit_application
  - 모듈 로드 성공: tools.finish_task_tool
    - 도구 등록 완료: finish_task
  - 모듈 로드 성공: tools.synchronize_knowledge_base_tool
    - 도구 등록 완료: synchronize_knowledge_base
  - 모듈 로드 성공: tools.fetch_document_from_mcp_tool
    - 도구 등록 완료: fetch_document_from_mcp
  - 모듈 로드 성공: tools.validate_document_tool
    - 도구 등록 완료: validate_document
[AI Agent] 현재 사용 가능한 도구: ['verify_business_registration', 'submit_application', 'finish_task', 'synchronize_knowledge_base', 'fetch_document_from_mcp', 'validate_document']


In [ ]:
# 에이전트에게 질문하기
# LLM 도구에 정의되어 있는 임의의 MCP 도구를 이용해서 신청을 진행하는 시나리오
agent.run("우리 가게 운영자금이 필요한데 소상공인 직접대출 신청 좀 해줘.")

tool_loader : <__main__.ToolLoader object at 0x79d7b440fe50>
tools : [<tools.verify_business_registration_tool.VerifyBusinessRegistrationTool object at 0x79d8585c2350>, <tools.submit_application_tool.SubmitApplicationTool object at 0x79d7b4289e90>, <tools.finish_task_tool.FinishTaskTool object at 0x79d7b43a4650>, <tools.synchronize_knowledge_base_tool.SynchronizeKnowledgeBaseTool object at 0x79d7b443bf50>, <tools.fetch_document_from_mcp_tool.FetchDocumentFromMcpTool object at 0x79d7b4438590>, <tools.validate_document_tool.ValidateDocumentTool object at 0x79d7b441d4d0>]
정의된 Tool들 : [{'type': 'function', 'function': {'name': 'verify_business_registration', 'description': '사용자의 사업자등록 상태가 유효한지 국세청 API로 확인합니다.', 'parameters': {'type': 'object', 'properties': {'user': {'type': 'object'}}, 'required': ['user']}}}, {'type': 'function', 'function': {'name': 'submit_application', 'description': '모든 검증된 서류를 모아 최종 목적지에 제출합니다.', 'parameters': {'type': 'object', 'properties': {'doc_tokens': {'type':

In [ ]:
# 에이전트에게 질문하기
# LLM 도구에 정의되어 있는 임의의 MCP 도구를 이용해서 신청을 진행하는 시나리오
agent.run("청년도약계좌를 만들고 싶어. 어떻게 해야하지?")

tool_loader : <__main__.ToolLoader object at 0x79d7b440fe50>
tools : [<tools.verify_business_registration_tool.VerifyBusinessRegistrationTool object at 0x79d8585c2350>, <tools.submit_application_tool.SubmitApplicationTool object at 0x79d7b4289e90>, <tools.finish_task_tool.FinishTaskTool object at 0x79d7b43a4650>, <tools.synchronize_knowledge_base_tool.SynchronizeKnowledgeBaseTool object at 0x79d7b443bf50>, <tools.fetch_document_from_mcp_tool.FetchDocumentFromMcpTool object at 0x79d7b4438590>, <tools.validate_document_tool.ValidateDocumentTool object at 0x79d7b441d4d0>]
정의된 Tool들 : [{'type': 'function', 'function': {'name': 'verify_business_registration', 'description': '사용자의 사업자등록 상태가 유효한지 국세청 API로 확인합니다.', 'parameters': {'type': 'object', 'properties': {'user': {'type': 'object'}}, 'required': ['user']}}}, {'type': 'function', 'function': {'name': 'submit_application', 'description': '모든 검증된 서류를 모아 최종 목적지에 제출합니다.', 'parameters': {'type': 'object', 'properties': {'doc_tokens': {'type':

In [ ]:
# 에이전트에게 질문하기
# 혁신성장 지원평가 자금대출
# Tool 이 없다고 판단하는 경우 새로운 Tool.py 를 만드는 과정
agent.run("내가 이번에 IT 스타트업을 차릴거야. 혁신성장 지원평가 받을 수 있는 대출을 알려줘.")

tool_loader : <__main__.ToolLoader object at 0x79d7b440fe50>
tools : [<tools.verify_business_registration_tool.VerifyBusinessRegistrationTool object at 0x79d8585c2350>, <tools.submit_application_tool.SubmitApplicationTool object at 0x79d7b4289e90>, <tools.finish_task_tool.FinishTaskTool object at 0x79d7b43a4650>, <tools.synchronize_knowledge_base_tool.SynchronizeKnowledgeBaseTool object at 0x79d7b443bf50>, <tools.fetch_document_from_mcp_tool.FetchDocumentFromMcpTool object at 0x79d7b4438590>, <tools.validate_document_tool.ValidateDocumentTool object at 0x79d7b441d4d0>]
정의된 Tool들 : [{'type': 'function', 'function': {'name': 'verify_business_registration', 'description': '사용자의 사업자등록 상태가 유효한지 국세청 API로 확인합니다.', 'parameters': {'type': 'object', 'properties': {'user': {'type': 'object'}}, 'required': ['user']}}}, {'type': 'function', 'function': {'name': 'submit_application', 'description': '모든 검증된 서류를 모아 최종 목적지에 제출합니다.', 'parameters': {'type': 'object', 'properties': {'doc_tokens': {'type':

In [ ]:
# 에이전트에게 질문하기
# TfIdf 의 한계로 검색어 유사도가 낮으면 제대로 된 답변을 하지 못한다.
# semantic 기반 검색엔진을 벡터DB에 도입하여 정확도를 높일 수 있다
# semantic 기반 검색엔진의 일반화로 잘못된 답변을 도출하는 경우 정밀도가 낮아질 수 있고, tfidf+semantic 혼합한 hybrid 검색을 이용
# Prompt engineering 을 통해 정확도 높은 검색 엔진 개발
agent.run("내가 이번에 IT 스타트업을 차릴거야. 사업자금을 만들고 싶은데 나만의 기발한 기술로 받을 수 있는 대출을 알려줘.")

tool_loader : <__main__.ToolLoader object at 0x79d7b440fe50>
tools : [<tools.verify_business_registration_tool.VerifyBusinessRegistrationTool object at 0x79d8585c2350>, <tools.submit_application_tool.SubmitApplicationTool object at 0x79d7b4289e90>, <tools.finish_task_tool.FinishTaskTool object at 0x79d7b43a4650>, <tools.synchronize_knowledge_base_tool.SynchronizeKnowledgeBaseTool object at 0x79d7b443bf50>, <tools.fetch_document_from_mcp_tool.FetchDocumentFromMcpTool object at 0x79d7b4438590>, <tools.validate_document_tool.ValidateDocumentTool object at 0x79d7b441d4d0>]
정의된 Tool들 : [{'type': 'function', 'function': {'name': 'verify_business_registration', 'description': '사용자의 사업자등록 상태가 유효한지 국세청 API로 확인합니다.', 'parameters': {'type': 'object', 'properties': {'user': {'type': 'object'}}, 'required': ['user']}}}, {'type': 'function', 'function': {'name': 'submit_application', 'description': '모든 검증된 서류를 모아 최종 목적지에 제출합니다.', 'parameters': {'type': 'object', 'properties': {'doc_tokens': {'type':

In [ ]:
# 에이전트에게 질문하기
# TfIdf 의 한계로 검색어 유사도가 낮으면 제대로 된 답변을 하지 못한다.
# semantic 기반 검색엔진을 벡터DB에 도입하여 정확도를 높일 수 있다
# semantic 기반 검색엔진의 일반화로 잘못된 답변을 도출하는 경우 정밀도가 낮아질 수 있고, tfidf+semantic 혼합한 hybrid 검색을 이용
# Prompt engineering 을 통해 정확도 높은 검색 엔진 개발
agent.run("편의점을 운영하는데 긴급하게 현금 융통이 필요해. 소상공인진흥공단에 받을 수 있는 정책이 없을까?")

tool_loader : <__main__.ToolLoader object at 0x0000029B1F597A10>
tools : [<tools.fetch_document_from_mcp_tool.FetchDocumentFromMcpTool object at 0x0000029B1F597B60>, <tools.search_knowledge_base_tool.SearchKnowledgeBaseTool object at 0x0000029B1F5978C0>, <tools.submit_application_tool.SubmitApplicationTool object at 0x0000029B1F597CB0>, <tools.synchronize_knowledge_base_tool.SynchronizeKnowledgeBaseTool object at 0x0000029B1F597E00>, <tools.validate_document_tool.ValidateDocumentTool object at 0x0000029B1F744050>, <tools.verify_business_registration_tool.VerifyBusinessRegistrationTool object at 0x0000029B1F7441A0>]
정의된 Tool들 : [{'type': 'function', 'function': {'name': 'fetch_document_from_mcp', 'description': '필요한 서류를 MCP를 통해 기관에서 가져옵니다.', 'parameters': {'type': 'object', 'properties': {'document_name': {'type': 'string', 'description': '가져올 서류의 정확한 이름'}, 'user_id': {'type': 'string', 'description': '요청하는 사용자의 ID'}}, 'required': ['doc_token', 'issue_date_str']}}}, {'type': 'function',

In [ ]:
agent.run("낚시를 잘 하는 법을 알려줘.")

tool_loader : <__main__.ToolLoader object at 0x0000029B1F597A10>
tools : [<tools.fetch_document_from_mcp_tool.FetchDocumentFromMcpTool object at 0x0000029B1F597B60>, <tools.search_knowledge_base_tool.SearchKnowledgeBaseTool object at 0x0000029B1F5978C0>, <tools.submit_application_tool.SubmitApplicationTool object at 0x0000029B1F597CB0>, <tools.synchronize_knowledge_base_tool.SynchronizeKnowledgeBaseTool object at 0x0000029B1F597E00>, <tools.validate_document_tool.ValidateDocumentTool object at 0x0000029B1F744050>, <tools.verify_business_registration_tool.VerifyBusinessRegistrationTool object at 0x0000029B1F7441A0>]
정의된 Tool들 : [{'type': 'function', 'function': {'name': 'fetch_document_from_mcp', 'description': '필요한 서류를 MCP를 통해 기관에서 가져옵니다.', 'parameters': {'type': 'object', 'properties': {'document_name': {'type': 'string', 'description': '가져올 서류의 정확한 이름'}, 'user_id': {'type': 'string', 'description': '요청하는 사용자의 ID'}}, 'required': ['doc_token', 'issue_date_str']}}}, {'type': 'function',

In [ ]:
agent.run("2025년 7월 21일 현재 파리로 가는 가장 싼 비행기를 알려줘.")

tool_loader : <__main__.ToolLoader object at 0x0000029B1F597A10>
tools : [<tools.fetch_document_from_mcp_tool.FetchDocumentFromMcpTool object at 0x0000029B1F597B60>, <tools.search_knowledge_base_tool.SearchKnowledgeBaseTool object at 0x0000029B1F5978C0>, <tools.submit_application_tool.SubmitApplicationTool object at 0x0000029B1F597CB0>, <tools.synchronize_knowledge_base_tool.SynchronizeKnowledgeBaseTool object at 0x0000029B1F597E00>, <tools.validate_document_tool.ValidateDocumentTool object at 0x0000029B1F744050>, <tools.verify_business_registration_tool.VerifyBusinessRegistrationTool object at 0x0000029B1F7441A0>]
정의된 Tool들 : [{'type': 'function', 'function': {'name': 'fetch_document_from_mcp', 'description': '필요한 서류를 MCP를 통해 기관에서 가져옵니다.', 'parameters': {'type': 'object', 'properties': {'document_name': {'type': 'string', 'description': '가져올 서류의 정확한 이름'}, 'user_id': {'type': 'string', 'description': '요청하는 사용자의 ID'}}, 'required': ['doc_token', 'issue_date_str']}}}, {'type': 'function',